In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
import pandas as pd
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.utils import to_categorical
print(tf.__version__)

In [ ]:
trainRawData = pd.read_csv('../input/digit-recognizer/train.csv')
testRawData = pd.read_csv('../input/digit-recognizer/test.csv')

train_label = trainRawData['label']
train_label = to_categorical(train_label)

train_data = (trainRawData.iloc[:, 1:].values.astype('float32') / 255.0 ).reshape(-1, 28, 28, 1)
test_data = (testRawData.iloc[:,:].values.astype('float32') / 255.0 ).reshape(-1, 28, 28, 1)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
def show_as_img(data,label):
    ax = plt.subplot(3, 3, 1)
    plt.imshow(data)
    plt.title(label)
    plt.axis("off")
    
show_as_img((trainRawData.iloc[1:2, 1:].values.astype('uint8')).reshape(28, 28, 1), trainRawData['label'][1] )

In [ ]:
from sklearn.model_selection import train_test_split
train_input, val_input, train_label, val_label = train_test_split(train_data, train_label, shuffle=True, test_size = 0.1, stratify = train_label)
#stratify: 데이터가 쏠려서 분배되는 것 방지

In [ ]:
model_1 = tf.keras.Sequential([
  tf.keras.layers.Conv2D(32, 3, input_shape=(28, 28, 1), activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128,  activation=tf.keras.layers.LeakyReLU(alpha=0.01)),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
model = model_1
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'])
# loss='categorical_crossentropy' -->원핫 인코딩(one-hot encoding) 시 사용하는 손실함수
# loss='sparse_categorical_crossentropy 
#-->정수값 [0,1,2], 단 multiclass multilabel classification같은 하나의 input이 여러 class에 속할 수 있는 경우 사용불가

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
history = model.fit(
  train_input, train_label,
  validation_data=(val_input, val_label),
  epochs=20,
  callbacks=[callback]
)

In [ ]:
model.summary()

In [ ]:
model.evaluate(val_input,val_label)

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
#i refered function and model from https://www.tensorflow.org/text/tutorials/text_classification_rnn 

In [ ]:
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

In [ ]:
x_test = testRawData.to_numpy().reshape(testRawData.shape[0], 28,28,1)
print(x_test.shape)

In [ ]:
prediction = model.predict(x_test)
prediction = np.argmax(prediction,axis=1)
print(prediction)

In [ ]:
submission=pd.read_csv('../input/digit-recognizer/sample_submission.csv')
submission['Label']=prediction
submission.to_csv('prediction4.csv', index=False)